# Особенности памяти в питоне

### Imports

In [5]:
import os
import sys
import random

## Общая информация

**Как это работает на уровне системы**

Программе для работы нужна память, чтобы хранить все свои объекты: сам код, состояние интепретатора (для питона), значения переменных и т.д. Однако изначально вся память принадлежит операционной системе, которая не склонна просто так ей делиться. Таким образом программе во время выполнения приходится делать запрос к ОС, чтобы попросить ту выделить нужное количество памяти. Эта операция очень дорогая с точки зрения времени, поэтому лучше делать ее пореже, однако просто один раз запросить много тоже нехорошо: остальной ОС тоже надо как-то выживать.

Отсюда появляется множество алгоритмов работы с памятью, большая часть из которых сводится к своевременному выделению (аллокации) и освобождению памяти.

Питон $-$ язык с реализованными механизмами автоматического управления памятью. Это означает, что вам не приходится (в отличие от того же c++) руками выделять память при создании нового объекта, питон это делает за вас. Для того, чтобы это работало, есть несколько механизмов:
1. Сама память внутри программы устроена следующим образом (арена обычно 256 кб, пул $-$ 4кб, а блоки бывают разные, но из набора 8, 16, 24, ..., 512 байт):  
![memory](https://habrastorage.org/getpro/habr/upload_files/e7f/514/64c/e7f51464c6998b75f4f3fa2bfe0bb5ef)
2. Счетчик ссылок: считает, сколько раз ссылаются на объект в программе. Когда ссылки заканчиваются, объект уничтожается. К увеличению кол-ва ссылок приводит:
    - присваивание
    - передача в функцию
    - вставка объекта в список
3. Сборщик мусора $-$ помогает решать задачу управления памятью там, где не справляется счетчик ссылок (при циклических ссылках)

In [3]:
a = [1, 3, 5]
b = [2, 5, 7]
a.append(b)
b.append(a)
a, b

([1, 3, 5, [2, 5, 7, [...]]], [2, 5, 7, [1, 3, 5, [...]]])

А еще мы можем смотреть, сколько памяти занимает написанная нами программа при выполнении. В этом нам помогает библиотека `memory_profiler`.

In [4]:
!pip install -U memory_profiler -q

Посмотрим, сколько памяти ест следующая функция (`@profile` $-$ декоратор, который говорит библиотеке, какую функцию мы хотим исследовать):
```python
@profile
def my_func():
    a = [1] * (10 ** 6)
    b = [2] * (2 * 10 ** 7)
    del b
    return a

if __name__ == '__main__':
    my_func()
```
Чтобы это сделать, нам надо положить код в отдельный файл (`func.py`) и запустить профайлинг как команду в терминале/командной строке. В выдаче будет несколько важных нам колонок:
- Mem usage $-$ сколько памяти использует интерпретатор питона после выполнения этой строки
- Increment $-$ разница в занимаемой памяти между текущей строкой и предыдущей

In [6]:
!python -m memory_profiler func.py

Filename: func.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     1   38.016 MiB   38.016 MiB           1   @profile
     2                                         def my_func():
     3   45.750 MiB    7.734 MiB           1       a = [1] * (10 ** 6)
     4  198.117 MiB  152.367 MiB           1       b = [2] * (2 * 10 ** 7)
     5   45.773 MiB -152.344 MiB           1       del b
     6   45.773 MiB    0.000 MiB           1       return a




## Ссылки с точки зрения кода

Питон хранит все данные по ссылке, иногда это создает проблемы в неожиданных местах. `id()` $-$ функция, которая позволяет получить индивидуальный номер объекта (а в питоне все объекты), то есть узнать, как питон ссылается на него в памяти

In [7]:
a = 0
b = 1
a_list = [0, b]
id(0), id(a), id(a_list[0])

(138149200576720, 138149200576720, 138149200576720)

In [8]:
a = 10*1000
id(10*1000), id(a)

(138148694974736, 138148694973296)

In [9]:
id(1), id(b), id(a_list[1])

(138149200576752, 138149200576752, 138149200576752)

А вот с более сложными типами будет интереснее. Какой ответ ожидается в ячейке ниже?

In [ ]:
test_lst1 = [0, 4, 7, 9]
test_lst2 = [2, 6, 3, 8]
test_lst3 = [0, 4, 7, 9]
id(test_lst1), id(test_lst2), id(test_lst3)

А теперь?

In [ ]:
test_lst4 = test_lst1
id(test_lst1), id(test_lst3), id(test_lst4)

Еще есть функция `sys.getrefcount`. Я слабо себе представляю, как она может пригодиться вам в жизни (если вы не занимаетесь хардкодом, но тогда вы скорее всего делаете это не на питоне...), но понять хранение питона точно поможет. Надо заметить, что эта функция внутри себя создает еще одну ссылку (возможно, для очень сложным структур больше, но тут я не уверена), так что полученное значение надо уменьшать на единицу

In [13]:
sys.getrefcount([])

1

In [14]:
sys.getrefcount(test_lst2)

3

In [15]:
sys.getrefcount(test_lst1)

5

In [16]:
sys.getrefcount(test_lst4)

5

А вот ниже мы, кажется, случайно увидели что-то очень для питона приватное: мы явно не ссылаемся на нули в таком количестве, значит, это информация о ссылках во внутренней струкутре питоне

In [17]:
sys.getrefcount(0), sys.getrefcount(True), sys.getrefcount(None)

(8376, 9196, 46889)

In [18]:
sys.getrefcount(1), sys.getrefcount(2), sys.getrefcount(2**10), sys.getrefcount(2**100)

(6139, 3090, 2, 1)

Итак, с более сложными струкутрами данных питон начинает мухлевать и экономить усилия, дублируя в памяти ссылки, а не сами объекты. Это может ускорить работу и сэкономить память, но провоцирует опасные ситуации. Не баг, а фича, но фича с подвохом.

Теперь о подвохах

In [19]:
test_lst1, test_lst3, test_lst4

([0, 4, 7, 9], [0, 4, 7, 9], [0, 4, 7, 9])

In [20]:
test_lst1.append(10)

Что мы ожидаем увидеть?

In [ ]:
test_lst1, test_lst3, test_lst4

Теперь о менее очевидном, но более опасном
</br>
Пусть надо написать функцию, которая считает сумму всех элементов в двух списках. Узнав о классной встроенной функции `sum()` мы решили, что просто сложим все элемерты в первый список и посчитаем сумму. Тут со стороны кажется, что сплошные плюсы: мы экономим строки кода, экономим место в переменных...

In [22]:
def test_func1(lst1, lst2):
    lst1 += lst2
    return sum(lst1)

In [23]:
test_func1(test_lst1, test_lst2)

49

Что будет выведено дальше?

In [ ]:
test_lst1, test_lst2

А тепреь еще интереснее

In [ ]:
test_lst4

Казалось бы, и в первом, и во втором случае мы его вообще не трогали, во втором мы еще и не сохраняли никуда результат, но питон все решил за нас.

Еще один пример, теперь про значения по умолчанию. Пусть нам надо написать функцию, которая получает на вход список пассажиров автобуса, имя пассажира и маркер того, что надо сделать: высадить или посадить. По умолчанию автобус пустой

In [26]:
def bus_func(bus=[], name=None, take=True):
    if name is not None:
        if take:
            bus.append(name)
        else:
            if name in bus:
                # высаживаем первого пассажира с таким именем... кому-то не повезет :)
                bus.remove(name)
    return bus

Проверяем, что все работает

In [27]:
my_bus = []
my_bus = bus_func(my_bus, 'Kate', True)
my_bus

['Kate']

In [28]:
my_bus = bus_func(my_bus, 'Max', True)
my_bus = bus_func(my_bus, 'Kate', False)
my_bus

['Max']

Посадим в пустой автобус кого-нибудь еще

In [29]:
my_bus2 = bus_func(name='Alice', take=True)
my_bus2 = bus_func(my_bus2, 'Peter', True)
my_bus2

['Alice', 'Peter']

Теперь в другой пустой автобус

In [30]:
my_bus3 = bus_func(name='John', take=True)
my_bus3

['Alice', 'Peter', 'John']

И сделаем просто пустой автобус. Вдруг пригодится

In [31]:
my_bus4 = bus_func()
my_bus4

['Alice', 'Peter', 'John']

Вспомним про второй автобус и высадим кого-нибудь оттуда

In [32]:
my_bus2 = bus_func(my_bus3, 'Peter', False)
my_bus2

['Alice', 'John']

А теперь посмотрим, что же произошло с функцией. У каждой функции есть атрибут `__defaults__`, в котором содержаться значения по умолчанию аргументов этой функции

In [33]:
bus_func.__defaults__

(['Alice', 'John'], None, True)

Ну, теперь это автобус, у которого есть пассажиры по умолчанию :)

**Почему так получается?**
Когда мы создаем объект в питоне (а функции и списки $-$ это все объекты, хотя и разных классов), по очереди вызываются два метода:
- `__new__`, который отвечает за создание объекта
- `__init__`, который отвечает за инициализацию объекта

Именно в них происходит выделение памяти и присваивание объекту его личного адреса, по которому питон будет его искать. Когда мы даем питону прочитать описание функции, мы на самом деле создаем новый объект функции, с которым потом будем работать. При создании в том числе создаются и ссылки на все параметры по умолчанию.

Из-за этого получается, что для всех вызовов функции это одни и те же объекты, что в случае со списками (и еще словарями, множествами и любыми другими изменяемыми типами данных) приводит к описанному выше поведению.

Давайте посмотрим на это на практике. Для этого создадим наш собственный класс списка, который от обычного будет отличаться только оповещением нас о своем создании.

In [34]:
class MyList(list):
    def __init__(self, *args, **kwargs):
        print('hello i am here')
        super().__init__(*args, **kwargs)

Теперь опишем функцию с нашим списком как параметром по умолчанию:

In [35]:
def bus_func_cust(bus=MyList(), name=None, take=True):
    if name is not None:
        if take:
            bus.append(name)
        else:
            if name in bus:
                bus.remove(name)
    return bus

hello i am here


Обратите внимание, что оповещение о создании пришло нам один раз при описании функции и больше не появляется

In [36]:
my_bus2 = bus_func_cust(name='Alice', take=True)
my_bus2 = bus_func_cust(my_bus2, 'Peter', True)
my_bus2

['Alice', 'Peter']

In [37]:
my_bus4 = bus_func_cust()
my_bus4

['Alice', 'Peter']

In [38]:
bus_func_cust.__defaults__

(['Alice', 'Peter'], None, True)

## Сравнения

Теперь с темы ссылок перейдем к сравнениям, они связаны. В питоне есть два способа проверить равенство двух объектов: </br>`a == b`</br>`a is b` </br>Давайте вернем наши чудесные списки

In [39]:
test_lst1 = [0, 4, 7, 9]
test_lst2 = [2, 6, 3, 8]
test_lst3 = [0, 4, 7, 9]
test_lst4 = test_lst1

Результаты какой пары из трех ячеек ниже совпадут?

In [ ]:
test_lst1 == test_lst2, test_lst1 == test_lst3, test_lst1 == test_lst4

In [ ]:
test_lst1 is test_lst2, test_lst1 is test_lst3, test_lst1 is test_lst4

In [ ]:
id(test_lst1) == id(test_lst2), id(test_lst1) == id(test_lst3), id(test_lst1) == id(test_lst4)

Данные операторы отличаются тем, что один сравнивает содерждание объектов, а другой $-$ их идентификаторы. Поэтому в большинстве случаев надо использовать `==`: нас чаще интересует содержание, а не странные питоновские нюансы памяти. Однако сравнение с уникальными сущностями типа `None`, `False` или `True`лучше производить через `is`. Если кто-то из вас пользуется пайчармом, то он мог говорить вам что-то такое

О проверке типа `if cat`, когда `cat` - это различные типы данных

In [43]:
def if_cat(cat):
    if cat:
        return 'yes'
    else:
        return 'no'

In [44]:
if_cat(True), if_cat(False)

('yes', 'no')

In [45]:
if_cat(None)

'no'

In [46]:
if_cat([]), if_cat(''), if_cat({}), if_cat(set())

('no', 'no', 'no', 'no')

In [47]:
if_cat(0), if_cat(1), if_cat(6)

('no', 'yes', 'yes')

In [48]:
if_cat([12, ]), if_cat('hi!'), if_cat({'hi!': 12})

('yes', 'yes', 'yes')

Где тут может быть подвох: дейсвтия на `None` часто требуется прописывать отдельно, и такая проверка может перепутать его с любым другим пустым объектом. Пусть у вас есть функция, в которую передается параметр, отвечающий за кол-во симовлов при выводе. Если он `None`, то вывести надо все

In [49]:
def test_func3(text, line=None):
    # do something
    if not line:
        return text
    else:
        return text[:line]

In [51]:
text = '''
Лиса предложила раку бегать наперегонки. Рак согласился. Лиса побежала, а рак уцепился за лисий хвост.
Лиса добежала до места. Обернулась лиса, а рак отцепился и говорит: «A я давно тут тебя жду».
'''

In [52]:
test_func3(text, 10)

'\nЛиса пред'

In [53]:
test_func3(text, None)

'\nЛиса предложила раку бегать наперегонки. Рак согласился. Лиса побежала, а рак уцепился за лисий хвост.\nЛиса добежала до места. Обернулась лиса, а рак отцепился и говорит: «A я давно тут тебя жду».\n'

In [54]:
test_func3(text, 0)

'\nЛиса предложила раку бегать наперегонки. Рак согласился. Лиса побежала, а рак уцепился за лисий хвост.\nЛиса добежала до места. Обернулась лиса, а рак отцепился и говорит: «A я давно тут тебя жду».\n'

Мы же не хотели получать символы в выводе, ввели ноль, а тут вдруг целый текст. Нехорошо :)</br>
Поэтому лучше делать сравнение с `None` явным

```python
if line is None:
    ...
````